<a href="https://colab.research.google.com/github/falanga/jordana-move-audio/blob/main/TentandoEntenderOErro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/code/amlanpraharaj/random-forest-using-mfcc-features/notebook

https://www.kaggle.com/code/anmour/svm-using-mfcc-features/notebook

https://www.kaggle.com/code/ilyamich/mfcc-implementation-and-tutorial

# Criar base de dados para Treinamento e Testes Com e Sem Inteligibilidades

In [ ]:
# Salva dados num arquivo
def salva_dados(dados, nome_do_arquivo):
  from google.colab import drive
  diretorio_google = '/content/drive'
  drive.mount(diretorio_google, True)

  from pathlib import Path  
  filepath = Path(diretorio_google + "/MyDrive/Jordana/" + nome_do_arquivo)  

  filepath.parent.mkdir(parents=True, exist_ok=True)  
  dados.to_csv(filepath)
  print("Arquivo {} salvo", nome_do_arquivo)

In [ ]:
#Funão para Gerar os dados MFCC

from scipy.stats import skew

# Generate mfcc features with mean and standard deviation
def get_mfcc(nome_do_audio):
    data, SAMPLE_RATE = librosa.load(nome_do_audio)

    try:
        ft1 = librosa.feature.mfcc(data, sr = SAMPLE_RATE, n_mfcc=13)
        # ft2 = librosa.feature.zero_crossing_rate(data)[0]
        # ft3 = librosa.feature.spectral_rolloff(data)[0]
        # ft4 = librosa.feature.spectral_centroid(data)[0]
        # ft5 = librosa.feature.spectral_contrast(data)[0]
        # ft6 = librosa.feature.spectral_bandwidth(data)[0]
        ft1_trunc = np.hstack((np.mean(ft1, axis=1), np.std(ft1, axis=1), skew(ft1, axis = 1), np.max(ft1, axis = 1), np.median(ft1, axis = 1), np.min(ft1, axis = 1)))
        # ft2_trunc = np.hstack((np.mean(ft2), np.std(ft2), skew(ft2), np.max(ft2), np.median(ft2), np.min(ft2)))
        # ft3_trunc = np.hstack((np.mean(ft3), np.std(ft3), skew(ft3), np.max(ft3), np.median(ft3), np.min(ft3)))
        # ft4_trunc = np.hstack((np.mean(ft4), np.std(ft4), skew(ft4), np.max(ft4), np.median(ft4), np.min(ft4)))
        # ft5_trunc = np.hstack((np.mean(ft5), np.std(ft5), skew(ft5), np.max(ft5), np.median(ft5), np.min(ft5)))
        # ft6_trunc = np.hstack((np.mean(ft6), np.std(ft6), skew(ft6), np.max(ft6), np.median(ft6), np.max(ft6)))
        # return pd.Series(np.hstack((ft1_trunc, ft2_trunc, ft3_trunc, ft4_trunc, ft5_trunc, ft6_trunc)))
        return pd.Series(np.hstack((ft1_trunc)))

    except:
        print('bad file')
        return pd.Series([0]*210)


# Função para identificar a inteligibilidade dos áudios

def get_inteligibilidade(nome_do_audio):
  very_low_list = ['M01', 'M04', 'M12', 'F03']
  low_list = ['M06', 'M07', 'F02', 'M16']
  mid_list = ['M05', 'F04', 'M11']
  high_list = ['M08', 'M09', 'M10', 'F04', 'M14']

  nome = nome_do_audio.split(os.sep)[-1]
  inteligibilidade = nome.split('_')[0]
  if inteligibilidade in very_low_list:
    return 'very_low'
  if inteligibilidade in low_list:
    return 'low'
  if inteligibilidade in mid_list:
    return 'mid'
  if inteligibilidade in high_list:
    return 'high'
  return 'control'

# Gera dados de treinamento, palavra por palavra
import os
import time as tm ###
import librosa

import glob
import numpy as np
from IPython.display import Audio
from sklearn.model_selection import train_test_split ###

import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()

import librosa.display as ld
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive', True)

# Definição de variáveis
dir_de_sons = '/content/drive/MyDrive/Jordana/juntos/'

print("⚙️ Extração MFCC para as palavras")
start = tm.time()

# preenche a lista de nomes dos arquivos
labels_list = []
for name in glob.glob(str(dir_de_sons) + '/*'):
  labels_list.append(name.split(os.sep)[-1])


# labels_list é a lista de palavras
speech_labels_list = []
speech_inteligibilidade_list = []
speech_data_list = []

for label in labels_list:
  # label é a palavras sendo dita
  i = 0
  for name in glob.glob(str(dir_de_sons) + label + '/*'):
    speech_data_list.append(name)
    speech_labels_list.append(label)
    speech_inteligibilidade_list.append(get_inteligibilidade(name))
    # i += 1
    # if i == 300:
    #   break

data = pd.DataFrame()
data['fname'] = speech_data_list

data = data['fname'].progress_apply(get_mfcc)
data['fname'] = speech_data_list
data['label'] = speech_labels_list
data['inteligibilidade'] = speech_inteligibilidade_list

Mounted at /content/drive
⚙️ Extração MFCC para as palavras


  0%|          | 0/751 [00:00<?, ?it/s]

In [ ]:
#Salvar dados que serão utilizados no treinamento
salva_dados(data, "com_inteligibilidades.csv")
dados_sem = data.drop('inteligibilidade', axis=1)
salva_dados(dados_sem, "sem_inteligibilidades.csv")


Mounted at /content/drive
Arquivo {} salvo com_inteligibilidades.csv
Mounted at /content/drive
Arquivo {} salvo sem_inteligibilidades.csv


# Gerar dados de treinamento e teste 

In [ ]:
# LER DADOS PARA UTILIZAÇÃO EM TREINAMENTO E TESTE
import os
import time as tm ###
import pandas as pd

def ler_dados(nome_do_arquivo):
  from google.colab import drive
  diretorio_google = '/content/drive'
  drive.mount(diretorio_google, True)

  from pathlib import Path  
  filepath = Path(diretorio_google + "/MyDrive/Jordana/" + nome_do_arquivo)  

  filepath.parent.mkdir(parents=True, exist_ok=True)  
  return pd.read_csv(filepath)


# `Fazendo para os dados SEM inteligibilidade`

In [ ]:
# Leitura dos dados sem inteligibilidade
import numpy as np

dados = ler_dados("com_inteligibilidades.csv")

speech_labels_list = dados['label']

dados_sem=dados.drop('inteligibilidade', axis=1)

# Separar dados de treinamento
from sklearn.model_selection import train_test_split ###
X_train, X_test_ret, y_train, y_test_ret = train_test_split(dados_sem, speech_labels_list, test_size=0.3)

X_test_ret['label'] = np.zeros((len(X_test_ret)))



Mounted at /content/drive


In [ ]:
# Functions from Random Foresth using MFCC ttps://www.kaggle.com/amlanpraharaj/random-forest-using-mfcc-features
# Construct features set
X = X_train.drop(['label', 'fname'], axis=1)
feature_names = list(X.columns)
X = X.values
labels = np.sort(np.unique(X_train.label.values))
num_class = len(labels)
c2i = {}
i2c = {}
for i, c in enumerate(labels):
    c2i[c] = i
    i2c[i] = c
y = np.array([c2i[x] for x in X_train.label.values])

In [ ]:
X_test = X_test_ret.drop(['label', 'fname'], axis=1)
X_test = X_test.values
print(X.shape)
print(X_test.shape)

(525, 79)
(226, 79)


In [ ]:
##Divisão dos dados
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=10, shuffle = True)

In [ ]:
#fitting random forest on the dataset
import time as tm ###
start = tm.time()

rfc = RandomForestClassifier(n_estimators = 150)
rfc.fit(X_train, y_train)
def proba2labels(preds, i2c, k=3):
    ans = []
    ids = []
    for p in preds:
        idx = np.argsort(p)[::-1]
        ids.append([i for i in idx[:k]])
        ans.append(' '.join([i2c[i] for i in idx[:k]]))

    return ans, ids
#checking the accuracy of the model
print(rfc.score(X_val, y_val))

end = tm.time()
print("✅ Concluído. Execução: {} segundos.".format(round(end-start,2)))

0.9936708860759493
✅ Concluído. Execução: 0.37 segundos.


In [ ]:
###Salvar os melhores parâmetros do Random Forest
import time as tm ###
start = tm.time()

import pandas as pd

data_best_params = pd.DataFrame()
# diction = grid.best_params_
# diction = {'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}
diction = {'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}

data_best_params = data_best_params.append( diction, ignore_index=True)

#Salvar dados de treinamento
from google.colab import drive
diretorio_google = '/content/drive'
drive.mount(diretorio_google, True)

from pathlib import Path  
filepath = Path(diretorio_google + "/MyDrive/Jordana/" + "best_params_100_words_knn_RF_sem_inteligibilidade.csv")  
filepath.parent.mkdir(parents=True, exist_ok=True)  
data_best_params.to_csv(filepath)

end = tm.time()
print("✅ Concluído. Execução: {} segundos.".format(round(end-start,2)))  

Mounted at /content/drive
✅ Concluído. Execução: 4.84 segundos.


In [ ]:
data_best_params.head()

,criterion,max_depth,min_samples_leaf,min_samples_split,n_estimators
0,entropy,5.0,2.0,2.0,200.0


# `Fazendo para os dados COM inteligibilidade`

In [ ]:
from pandas.core.arrays.categorical import pd_array
# Função para separar dados de teste e de treinamento especial para excluir alguns itens
import random

def random_unique(lista, quantidade):
  retorno = []

  if quantidade < len(lista):
    while len(retorno)<quantidade:
      escolhido = random.choice(lista)
      if escolhido not in retorno:
        retorno.append(escolhido)
    return retorno
  return lista

def drop_others(lista, preserve_list):
  retorno = lista
  todos = list(range(len(lista)))
  todos = list(set(todos) - set(preserve_list))
  for i in todos:
    retorno = retorno.drop(i)
  print("Tamanho de retorno ", len(retorno))
  return retorno


def train_test_split_dysarthric(X, inteligibilidades, test_size=0.3):
  falas_por_inteligibilidade = {}
  falas_por_inteligibilidade_train = {}
  falas_por_inteligibilidade_test = {}

  falas_por_inteligibilidade_train = []

  # Cria o dicionário com as inteligibilidades
  for inteligibilidade in inteligibilidades:
    falas_por_inteligibilidade[inteligibilidades[inteligibilidade]] = []
    falas_por_inteligibilidade_test[inteligibilidades[inteligibilidade]] = []

  # Cria os índices para cada fala de acordo com sua inteligibilidade
  i = range(len(X))
  for index in i:
     inteligibilidade_da_fala = X.iloc[index]['inteligibilidade']
     if inteligibilidade_da_fala in inteligibilidades:
        indice_da_inteligibilidade_da_fala = inteligibilidade_list[inteligibilidade_da_fala]
        falas_por_inteligibilidade[indice_da_inteligibilidade_da_fala].append(index)

  # Define os testes para cada inteligibilidade de acordo com o parâmetro passado
  for inteligibilidade_indice in inteligibilidades:
    indice = inteligibilidades[inteligibilidade_indice]
    # Define a quantidade de testes para a inteligibilidade
    n_test = (int) (len(falas_por_inteligibilidade[indice]) * test_size)
    print("Vamos fazer ", n_test, " testes para a inteligibilidade ", inteligibilidade_indice)
    # Recupera indices sorteados para teste pra a inteligibilidade atual
    falas_por_inteligibilidade_test[indice] = random_unique(falas_por_inteligibilidade[indice], n_test)

  # Seleciona as falas que serão utilizadas para teste para separar as de treinamento
  falas_para_teste = []
  i = range(len(falas_por_inteligibilidade_test))
  for index in i:
    falas_para_teste = falas_para_teste + falas_por_inteligibilidade_test[index]
  # As falas para treino são aquelas não utilizadas para testes
  falas_para_treino = list(set(range(len(X))) - set(falas_para_teste))

  print("Decidindo os treinos para um total de testes = ", len(falas_para_teste))
  ret_falas_para_treino = drop_others(X, falas_para_treino)
  ret_falas_para_treino = ret_falas_para_treino.drop('inteligibilidade', axis=1)

  ret_falas_para_teste = []
  
  for inteligibilidade_indice in inteligibilidades:
    indice = inteligibilidades[inteligibilidade_indice]
    print("Decidindo os testes")
    todos = list(range(len(X)))
    lista_de_teste = drop_others(X, falas_por_inteligibilidade_test[indice])
    ret_falas_para_teste.append(lista_de_teste.drop('inteligibilidade', axis=1))

  y_train = pd.DataFrame(ret_falas_para_treino, columns = ['label'])

  y_train = y_train.squeeze()

  y_test = []

  for i in range(len(ret_falas_para_teste)):
    y_ = pd.DataFrame(ret_falas_para_treino, columns = ['label'])
    y_test.append(y_.squeeze())

  return ret_falas_para_treino, ret_falas_para_teste, y_train, y_test

In [ ]:
# Leitura dos dados com inteligibilidade
import numpy as np

dados = ler_dados("com_inteligibilidades.csv")

speech_labels_list = dados['label']

#Recupera os dados de treinamento e teste:
inteligibilidade_list = {'very_low':0, 'low':1,'mid':2,'high':3, 'control': 4}

X_train, X_test_ret, y_train, y_test_ret  = train_test_split_dysarthric(dados, inteligibilidade_list)

for i in range(len(X_test_ret) - 1):
  X_test_ret[i]['label'] = np.zeros((len(X_test_ret[i])))







Mounted at /content/drive
Vamos fazer  26  testes para a inteligibilidade  very_low
Vamos fazer  24  testes para a inteligibilidade  low
Vamos fazer  24  testes para a inteligibilidade  mid
Vamos fazer  33  testes para a inteligibilidade  high
Vamos fazer  117  testes para a inteligibilidade  control
Decidindo os treinos para um total de testes =  224
Tamanho de retorno  527
Decidindo os testes
Tamanho de retorno  26
Decidindo os testes
Tamanho de retorno  24
Decidindo os testes
Tamanho de retorno  24
Decidindo os testes
Tamanho de retorno  33
Decidindo os testes
Tamanho de retorno  117


In [ ]:
# Functions from Random Foresth using MFCC ttps://www.kaggle.com/amlanpraharaj/random-forest-using-mfcc-features
# Construct features set
X = X_train.drop(['label', 'fname'], axis=1)
feature_names = list(X.columns)
X = X.values
labels = np.sort(np.unique(X_train.label.values))
num_class = len(labels)
c2i = {}
i2c = {}
for i, c in enumerate(labels):
    c2i[c] = i
    i2c[i] = c
y = np.array([c2i[x] for x in X_train.label.values])

In [ ]:
X_test = []
for i in range(len(X_test_ret)):
  X_temp = X_test_ret[i].drop(['label', 'fname'], axis=1)
  X_test.append(X_temp.values)
  print(X.shape)
  print(X_test[i].shape)


(527, 79)
(26, 79)
(527, 79)
(24, 79)
(527, 79)
(24, 79)
(527, 79)
(33, 79)
(527, 79)
(117, 79)


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=10, shuffle = True)

In [ ]:
#fitting random forest on the dataset
import time as tm ###
start = tm.time()

rfc = RandomForestClassifier(n_estimators = 150)
rfc.fit(X_train, y_train)
def proba2labels(preds, i2c, k=3):
    ans = []
    ids = []
    for p in preds:
        idx = np.argsort(p)[::-1]
        ids.append([i for i in idx[:k]])
        ans.append(' '.join([i2c[i] for i in idx[:k]]))

    return ans, ids
#checking the accuracy of the model
print(rfc.score(X_val, y_val))

end = tm.time()
print("✅ Concluído. Execução: {} segundos.".format(round(end-start,2)))

1.0
✅ Concluído. Execução: 0.41 segundos.


In [ ]:
###Salvar os melhores parâmetros do Random Forest
import time as tm ###
start = tm.time()

import pandas as pd

data_best_params = pd.DataFrame()
# diction = grid.best_params_
# diction = {'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}
diction = {'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}

data_best_params = data_best_params.append( diction, ignore_index=True)

#Salvar dados de treinamento
from google.colab import drive
diretorio_google = '/content/drive'
drive.mount(diretorio_google, True)

from pathlib import Path  
filepath = Path(diretorio_google + "/MyDrive/Jordana/" + "best_params_100_words_knn_RF_sem_inteligibilidade.csv")  
filepath.parent.mkdir(parents=True, exist_ok=True)  
data_best_params.to_csv(filepath)

end = tm.time()
print("✅ Concluído. Execução: {} segundos.".format(round(end-start,2)))  

Mounted at /content/drive
✅ Concluído. Execução: 4.91 segundos.


In [ ]:
data_best_params.head()

,criterion,max_depth,min_samples_leaf,min_samples_split,n_estimators
0,entropy,5.0,2.0,2.0,200.0
